In [5]:
#PROFILE INFO

nick = ''
tag = ''
quantidade = ''

In [6]:
#FUNÇÕES

import pandas as pd
import requests 

api_key = ''

def get_summonerId(puuid):
    url = f'https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api_key}'
    response = requests.get(url)
    
    data = response.json()
    return data.get('id')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    base_url = f'https://{region}.api.riotgames.com/'

    if summonerId is not None:
        final_url = 'lol/summoner/v4/summoners/'
        print(base_url+final_url+summonerId+'?api_key='+api_key)
        response = requests.get(base_url+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        final_url = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(base_url+final_url+'?api_key='+api_key)
        
        return response.json()['puuid']


def get_match_history(api_key, puuid, region='americas', count=20):
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={count}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter o histórico de partidas: {response.status_code}")
    return []

puuid = get_puuid(gameName=nick, tagLine=tag)
summonerId = get_summonerId(puuid)

def get_match_data_from_id(region='americas',matchId=None):

    url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{matchId}'
    print(url+'?api_key=' + api_key)
    response = requests.get(url+'?api_key='+api_key)

    return response.json()

def get_participant_id(api_key, match_id, puuid, region='americas'):
    try:
        url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
        headers = {"X-Riot-Token": api_key}
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        match_details = response.json()
        participants = match_details['metadata']['participants']
        
        
        if puuid in participants:
            participant_id = participants.index(puuid) + 1
            riot_id = match_details['info']['participants'][participant_id - 1]['riotIdGameName']
            return participant_id, riot_id
        else:
            print(f"PUUID {puuid} não encontrado na partida {match_id}.")
            return None, None
    except requests.RequestException as e:
        print(f"Erro ao fazer a requisição para obter o participant ID: {e}")
    except Exception as e:
        print(f"Erro ao processar os detalhes do jogo: {e}")
    return None, None

def processed_match(match_json, puuid):

    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']
    primary = styles[0]
    secondary = styles[1]
    side_dict = {
        100: 'blue',
        200: 'red'
    }

    #DATA

    match_id = metadata['matchId']
    patch = info['gameVersion']
    queue_id = info['queueId']
    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    champ_name = player['championName']
    team_position = player['teamPosition']
    side = side_dict[player['teamId']]
    player_side = next((player['teamId'] for player in match_json['info']['participants'] if player['puuid'] == puuid), None)
    if player_side is None:
        return None
    win = player['win']
    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    kda = (kills + assists) / deaths if deaths > 0 else (kills + assists)
                
    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']
    gold_earned = player['goldEarned']
    gold_spent = player['goldSpent']
    team_gold = sum(player['goldEarned'] for player in match_json['info']['participants'] if player['teamId'] == player_side)
    turretPlatesTaken = player['challenges']['turretPlatesTaken']
    first_blood_kill = player['firstBloodKill']
    first_blood_assist = player['firstBloodAssist']
    solo_kills = player['challenges']['soloKills']
    kills_under_own_turret = player['challenges']['killsUnderOwnTurret']
    kill_participation = player['challenges'].get('killParticipation', 0)
    damage_per_minute = player['challenges']['damagePerMinute']
    total_damage_dealt = player['totalDamageDealtToChampions']
    if 'teamDamagePercentage' in player['challenges']:
        team_damage_percentage = player['challenges']['teamDamagePercentage']
    else:
        team_damage_percentage = None
    total_damage_taken = player['totalDamageTaken']
    damage_taken_on_team_percentage = player['challenges']['damageTakenOnTeamPercentage']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_healed = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_ally_jungle_minions_killed = player['totalAllyJungleMinionsKilled']
    total_enemy_jungle_minions_killed = player['totalEnemyJungleMinionsKilled']
    vision_score = player['visionScore']
    vision_score_per_minute = player['challenges']['visionScorePerMinute']
    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    detector_wards_placed = player['detectorWardsPlaced']
    first_tower_kill = player['firstTowerKill']
    first_tower_assist = player['firstTowerAssist']
    total_damage_structure = player.get('damageDealtToBuildings', 0)
    turretsTakenWithRiftHerald = player['challenges']['turretsTakenWithRiftHerald']
    team_dragon_kills = sum(player.get('dragonKills', 0) for player in match_json['info']['participants'] if player['teamId'] == player_side)
    team_baron_kills = player['challenges']['teamBaronKills']
    team_rift_herald_kills = player['challenges']['teamRiftHeraldKills']
    teamElderDragonKills = player['challenges']['teamElderDragonKills']
    objectives_stolen = player['objectivesStolen']
    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']
    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']
    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']
    primary_style = primary['style']
    secondary_style = secondary['style']
    offense = stats['offense']
    defense = stats['defense']
    flex = stats['flex']
    participant_id = player['participantId']
    champ_id = player['championId']
    summoner_id = player['summonerId']
    
    #DATA FRAME

    matchDF= pd.DataFrame({

        "ID da partida": [match_id],
        "Patch": [patch],
        "Queue": [queue_id],
        "riot_id" : [riot_id],
        "Tag": [riot_tag],
        "Campeão": [champ_name],
        "Lane": [team_position],
        "Side": [side],
        "Win": [win],
        "Kills": [kills],
        "Mortes": [deaths],
        "Assistências": [assists],
        "KDA": [kda],
        "Spell 1": [summoner_1_id],
        "Spell 2": [summoner_2_id],
        "Ouro adquirido": [gold_earned],
        "Ouro gasto": [gold_spent],
        "Ouro do time": [team_gold],
        "Plates destruídas": [turretPlatesTaken],
        "First Blood": [first_blood_kill],
        "First Blood Assist": [first_blood_assist],
        "Solo kills": [solo_kills],
        "Kills em baixo da propria torre": [kills_under_own_turret],
        "Participação de Kill": [kill_participation],
        "Dano por minuto": [damage_per_minute],
        "Dano causado": [total_damage_dealt],
        "Dano sofrido": [total_damage_taken],
        "Porcentagem de dano do time": [team_damage_percentage],
        "Porcentagem de dano sofrido no time": [damage_taken_on_team_percentage],
        "Total de dano protegido por escudo em aliados": [total_damage_shielded],
        "Total de cura em aliados": [total_damage_healed],
        "Minions farmados": [total_minions_killed],
        "Jungle farmada": [neutral_minions_killed],
        "Jungle aliada farmada": [total_ally_jungle_minions_killed],
        "Jungle inimiga farmada": [total_enemy_jungle_minions_killed],
        "Placar de visão": [vision_score],
        "Placar de visão por minuto": [vision_score_per_minute],
        "Wards colocadas": [wards_placed],
        "Pinks colocadas": [detector_wards_placed],
        "Wards destruídas": [wards_killed],
        "Primeira torre": [first_tower_kill],
        "Assistência primeira torre": [first_tower_assist],
        "Dano total em estruturas": [total_damage_structure],
        "Torres destruídas com arauto": [turretsTakenWithRiftHerald],  
        "Dragons": [team_dragon_kills],
        "Arautos": [team_rift_herald_kills],
        "Barons": [team_baron_kills],
        "Dragões anciões": [teamElderDragonKills],
        "Objetivos roubados": [objectives_stolen],
        "Remake": [early_surrender],
        "Surrender": [surrender],
        "Quando iniciou": [game_start_timestamp],
        "Quando acabou": [game_end_timestamp],
        "Item0": [item0],
        "Item1": [item1],
        "Item2": [item2],
        "Item3": [item3],
        "Item4": [item4],
        "Item5": [item5],
        "Item6": [item6],
        "perk_primary_style": [primary_style],
        "perk_secondary_style": [secondary_style],
        "perk_keystone": [primary_keystone],
        "perk_primary_row_1": [primary_perk_1],
        "perk_primary_row_2": [primary_perk_2],
        "perk_primary_row_3": [primary_perk_3],
        "perk_secondary_row_1": [secondary_perk_1],
        "perk_secondary_row_2": [secondary_perk_2],
        "perk_shard_offense": [offense],
        "perk_shard_flex": [flex],
        "perk_shard_defense": [defense],
        "participant_id" :[participant_id],
        "Puuid": [puuid],
        "ID do Jogador": [summoner_id],
        "ID do Campeão": [champ_id],

        })

    return matchDF



In [ ]:
# MAIN

match_ids = get_match_history(puuid=puuid, count=quantidade, api_key=api_key)

dfMain = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(matchId=match_id)
    matchDF = processed_match(game, puuid)
    dfMain = pd.concat([dfMain, matchDF])

pd.options.display.max_columns = 100

# CORREÇÕES E DETALHES
dfMain['Porcentagem de Ouro'] = dfMain['Ouro adquirido'] / dfMain['Ouro do time']
dfMain['Farm Total'] = dfMain['Minions farmados'] + dfMain['Jungle farmada']
dfMain['Quando iniciou'] = pd.to_datetime(dfMain['Quando iniciou'], unit='ms')
dfMain['Quando acabou'] = pd.to_datetime(dfMain['Quando acabou'], unit='ms')
dfMain['Data da partida'] = dfMain['Quando iniciou'].dt.strftime('%d/%m/%Y')
dfMain['Duração da partida'] = dfMain['Quando acabou'] - dfMain['Quando iniciou']
dfMain['Duração da partida'] = dfMain['Duração da partida'].apply(lambda x: f"{int(x.total_seconds() // 3600):02}:{int((x.total_seconds() % 3600) // 60):02}:{int(x.total_seconds() % 60):02}")
dfMain['Participação de First Blood'] = dfMain.apply(lambda row: True if row['First Blood'] or row['First Blood Assist'] else False, axis=1)
dfMain['Primeira torre'] = dfMain.apply(lambda row: True if row['Primeira torre'] or row['Assistência primeira torre'] else False, axis=1)
dfMain.drop(['First Blood', 'First Blood Assist', 'Quando iniciou', 'Quando acabou', 'Assistência primeira torre'], axis=1, inplace=True)
dfMain = dfMain[dfMain['Remake'] != True]
dfMain['Lane'] = dfMain['Lane'].replace({'JUNGLE': 'JNG', 'MIDDLE': 'MID', 'BOTTOM': 'ADC', 'UTILITY': 'SUP'})
dfMain['Patch'] = dfMain['Patch'].str.split('.').str[:2].str.join('.')
dfMain['KDA'] = dfMain['KDA'].astype(str).str.replace('.', ',')
dfMain['Participação de Kill'] = dfMain['Participação de Kill'].astype(str).str.replace('.', ',')
dfMain['Dano por minuto'] = dfMain['Dano por minuto'].astype(str).str.replace('.', ',')
dfMain['Porcentagem de dano do time'] = dfMain['Porcentagem de dano do time'].astype(str).str.replace('.', ',')
dfMain['Porcentagem de dano sofrido no time'] = dfMain['Porcentagem de dano sofrido no time'].astype(str).str.replace('.', ',')
dfMain['Placar de visão por minuto'] = dfMain['Placar de visão por minuto'].astype(str).str.replace('.', ',')
dfMain['Porcentagem de Ouro'] = dfMain['Porcentagem de Ouro'].astype(str).str.replace('.', ',')

# IMPORTANT URL
perk = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/pt_br/v1/perks.json"
perk_styles = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/pt_br/v1/perkstyles.json"
items = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/pt_br/v1/items.json"
spells = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/summoner-spells.json"
queue = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/pt_br/v1/queues.json"
champion_icon = "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/"
summoner_spells_base_url = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/'


# LEITURA E EXTRAÇÃO DE DADOS DO JSON
def json_extract(obj, key):
    arr = []
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr
    values = extract(obj, arr, key)
    return values

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
items_json = requests.get(items).json()
spells_json = requests.get(spells).json()
queue_json = requests.get(queue).json()

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')
perk_dict = dict(map(lambda i, j: (int(i), j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')
perk_styles_dict = dict(map(lambda i, j: (int(i), j), perk_styles_ids, perk_styles_names))

items_ids = json_extract(items_json, 'id')
items_names = json_extract(items_json, 'name')
items_dict = dict(map(lambda i, j: (int(i), j), items_ids, items_names))

spells_ids = json_extract(spells_json, 'id')
spells_names = json_extract(spells_json, 'name')
spells_dict = dict(map(lambda i, j: (int(i), j), spells_ids, spells_names))

queue_ids = json_extract(queue_json, 'id')
queue_names = json_extract(queue_json, 'name')
queue_dict = dict(map(lambda i, j: (int(i), j), queue_ids, queue_names))


# APLICANDO TRADUÇÃO
dfMain['perk_keystone'] = dfMain['perk_keystone'].replace(perk_dict)
dfMain['perk_primary_row_1'] = dfMain['perk_primary_row_1'].replace(perk_dict)
dfMain['perk_primary_row_2'] = dfMain['perk_primary_row_2'].replace(perk_dict)
dfMain['perk_primary_row_3'] = dfMain['perk_primary_row_3'].replace(perk_dict)
dfMain['perk_secondary_row_1'] = dfMain['perk_secondary_row_1'].replace(perk_dict)
dfMain['perk_secondary_row_2'] = dfMain['perk_secondary_row_2'].replace(perk_dict)
dfMain['perk_primary_style'] = dfMain['perk_primary_style'].replace(perk_styles_dict)
dfMain['perk_secondary_style'] = dfMain['perk_secondary_style'].replace(perk_styles_dict)
dfMain['perk_shard_offense'] = dfMain['perk_shard_offense'].replace(perk_dict)
dfMain['perk_shard_flex'] = dfMain['perk_shard_flex'].replace(perk_dict)
dfMain['perk_shard_defense'] = dfMain['perk_shard_defense'].replace(perk_dict)

dfMain['Spell 1'] = dfMain['Spell 1'].replace(spells_dict)
dfMain['Spell 2'] = dfMain['Spell 2'].replace(spells_dict)
dfMain['Queue'] = dfMain['Queue'].replace(queue_dict)


dfMain['Item0_name'] = dfMain['Item0'].replace(items_dict)
dfMain['Item1_name'] = dfMain['Item1'].replace(items_dict)
dfMain['Item2_name'] = dfMain['Item2'].replace(items_dict)
dfMain['Item3_name'] = dfMain['Item3'].replace(items_dict)
dfMain['Item4_name'] = dfMain['Item4'].replace(items_dict)
dfMain['Item5_name'] = dfMain['Item5'].replace(items_dict)
dfMain['Item6_name'] = dfMain['Item6'].replace(items_dict)


def get_champion_icon_url(champion_id):
    return f'{champion_icon}{champion_id}.png'

# APLICANDO URL DO ÍCONE DO CAMPEÃO
dfMain['Champion_icon'] = dfMain['ID do Campeão'].astype(str).apply(get_champion_icon_url)


def get_item_image_url(item_id, patch):
    
    return f'https://ddragon.leagueoflegends.com/cdn/{patch}.1/img/item/{item_id}.png'

# APLICANDO URL DA IMAGEM DO ITEM
for i in range(7): 
    dfMain[f'Item{i}_image'] = dfMain.apply(lambda row: get_item_image_url(row[f'Item{i}'], row['Patch']), axis=1)

def get_summoner_spell_icon_url(spell_name):
    spell_name = spell_name.lower().replace(' ', '_')
    icon_url = f'{summoner_spells_base_url}summoner_{spell_name}.png'

    # Verificar casos especiais
    if 'summoner_ignite' in icon_url:
        return 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summonerignite.png'
    elif 'summoner_barrier' in icon_url:
        return 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summonerbarrier.png'
    elif 'summoner_clarity' in icon_url:
        return 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summonermana.png'
    elif 'summoner_cleanse' in icon_url:
        return 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summoner_boost.png'
    elif 'summoner_ghost' in icon_url:
        return 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summoner_haste.png'
    else:
        return icon_url

# APLICANDO URL DO ÍCONE DO FEITIÇO DE INVOCADOR
dfMain['Spell 1 icon'] = dfMain['Spell 1'].apply(get_summoner_spell_icon_url)
dfMain['Spell 2 icon'] = dfMain['Spell 2'].apply(get_summoner_spell_icon_url)

dfMain